In [54]:
pip install pandas scikit-learn numpy



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install matplotlib



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from urllib.parse import urlparse
import numpy as np
from math import log2

In [57]:
import matplotlib.pyplot as plt

# Read the data

In [63]:
df = pd.read_csv('combined_df.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df

,url,label
0,www.usaozsazps.com/information,1
1,awrs.cl/wp-content/themes/form/bill.charged.html,1
2,ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...,1
3,wwxhajudjgwjklckvzgs7.firebaseapp.com/,1
4,roomclip.jp,0
...,...,...
91721,optikgeneral.com/Webmail/12/Webmail/webmail.php,1
91722,thegioididong.com,0
91723,pressks.com,0
91724,68.178.168.47/Empresarial/,1


In [42]:
# 2019 top most abused TLD 
abused_TLD = ['buzz', 'work', 'desi', 'ooo', 'gdn', 'nagoya', 'ryukyu', 'top', 'life', 'monster']

In [43]:
# letters, digits, plus, hyphen, and dots are the basic components
import ipaddress as ip

def count_plus(url):
    return url.count('+')

def count_hyphen(url):
    return url.count('-')

def count_dots(url):
    return url.count('.')

# delimeters are used for query delimeters
def count_delimeters(url):
    count = 0
    delimeters = [';', '_', '&', '=']
    for u in url:
        if u in delimeters:
            count += 1
    return url.count

# @ might be used for username:passwords
def count_at(url):
    return url.count('@')

# ? might be used for query
def count_question(url):
    return url.count('?')

# also known as sub dir
def count_slash(url):
    return url.count('/')

def count_doubleslash(url):
    return url.count('//')

def count_subdomain(sub):
    if not sub:
        return 0
    else:
        return len(sub.split('.'))

def is_ip(url):
    try:
        if ip.ip_address(url):
            return 1
    except:
            return 0
    

In [47]:
pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 2.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
from urllib.parse import urlparse
import  tldextract
def getFeatures(url, label): 
    result = []
    url = str(url)
    
    #add the url to feature set
    result.append(url)
    
    #parse the URL and extract the domain information
    path = urlparse(url)
    ext = tldextract.extract(url)
    
    #counting number of dots in subdomain    
    result.append(count_dots(ext.subdomain))
    
    #checking hyphen in domain   
    result.append(count_hyphen(path.netloc))
    
    #length of URL    
    result.append(len(url))
    
    #checking @ in the url    
    result.append(count_at(path.netloc))
    
    #checking presence of double slash    
    result.append(count_doubleslash(path.path))
    
    #Count number of subdir    
    result.append(count_slash(path.path))
    
    #number of sub domain    
    result.append(count_subdomain(ext.subdomain))
    
    #length of domain name    
    result.append(len(path.netloc))
    
    #count number of queries    
    result.append(len(path.query))
    
    #Adding domain information
    
    #if IP address is being used as a URL     
    result.append(is_ip(ext.domain))
    
    #presence of Suspicious_TLD
    result.append(1 if ext.suffix in abused_TLD else 0)
    
    
    result.append(str(label))
    return result

# featureSet = pd.DataFrame(columns=('url','no of dots in subdomain','no of hyphen in domain','len of url','no of at in domain',\
# 'no of double slash','no of slash/subdir','no of subdomain','len of domain name','no of queries','is IP','presence of Suspicious_TLD','label'))

# for i in range(len(df)):
#     features = getFeatures(df["url"].loc[i], df["label"].loc[i])
#     featureSet.loc[i] = features

In [51]:
# featureSet.head()

,url,no of dots in subdomain,no of hyphen in domain,len of url,no of at in domain,no of double slash,no of slash/subdir,no of subdomain,len of domain name,no of queries,is IP,presence of Suspicious_TLD,label
0,www.usaozsazps.com/information,0,0,30,0,0,1,1,0,0,0,0,1
1,awrs.cl/wp-content/themes/form/bill.charged.html,0,0,48,0,0,4,0,0,0,0,0,1
2,ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...,1,0,88,0,0,2,2,0,0,0,0,1
3,wwxhajudjgwjklckvzgs7.firebaseapp.com/,0,0,38,0,0,1,1,0,0,0,0,1
4,roomclip.jp,0,0,11,0,0,0,0,0,0,0,0,0


In [70]:
from urllib.parse import urlparse

# Helper function to calculate the number of times each special character appears in the URL
def count_chars(url, char_list):
    return {f"num_{char}_url": url.count(char) for char in char_list}

# Function to check if an email is present in the URL
def check_email_in_url(url):
    return 1 if "@" in url else 0

# Calculate the length of the top-level domain (TLD)
def tld_length(url):
    domain = urlparse(url).netloc
    try:
        # Assuming the last part after the last dot is the TLD
        return len(domain.split('.')[-1])
    except IndexError:
        return 0

# Update the function to process a DataFrame of URLs
def extract_features_from_urls(df):
    def get_features(url):
        special_chars = ['.', '-', '_', '/', '?', '=', '@', '&', '!', ' ', '~', ',', '+', '*', '#', '$', '%']
        features = count_chars(url, special_chars)
        features['length_url'] = len(url)
        features['email_in_url'] = check_email_in_url(url)
        features['num_tld_url'] = tld_length(url)
        return features
    
    # Apply the get_features function to each URL in the DataFrame
    features_df = df['url'].apply(lambda x: pd.Series(get_features(x)))
    return pd.concat([df, features_df], axis=1)

In [72]:
df_features = extract_features_from_urls(df)
df_features

,url,label,num_._url,num_-_url,num___url,num_/_url,num_?_url,num_=_url,num_@_url,num_&_url,...,num_~_url,"num_,_url",num_+_url,num_*_url,num_#_url,num_$_url,num_%_url,length_url,email_in_url,num_tld_url
0,www.usaozsazps.com/information,1,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,30,0,0
1,awrs.cl/wp-content/themes/form/bill.charged.html,1,3,1,0,4,0,0,0,0,...,0,0,0,0,0,0,0,48,0,0
2,ipfs.eth.aragon.network/ipfs/bafybeifg3yzh6ekg...,1,3,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,88,0,0
3,wwxhajudjgwjklckvzgs7.firebaseapp.com/,1,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,38,0,0
4,roomclip.jp,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91721,optikgeneral.com/Webmail/12/Webmail/webmail.php,1,2,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,47,0,0
91722,thegioididong.com,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17,0,0
91723,pressks.com,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,0
91724,68.178.168.47/Empresarial/,1,3,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,26,0,0


In [30]:
X = df_features.drop(['url', 'label'], axis=1)
y = df_features['label']


# Assuming X is your features and y is your labels
# Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# If you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)  # 0.25 x 0.8 = 0.2

# Feature selection
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train, y_train)

selector = SelectFromModel(forest, prefit=True)
X_important_train = selector.transform(X_train)
X_important_val = selector.transform(X_val)
X_important_test = selector.transform(X_test)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/python/3.10.13/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/python/3.10.13/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [31]:
# Get a mask, or integer index, of the features selected
selected_features_mask = selector.get_support()

# Now, print the names of the features selected
selected_feature_names = X_train.columns[selected_features_mask]
print("Selected Features:")
print(selected_feature_names.tolist())

Selected Features:
['length', 'num_special_chars']


In [21]:
# Classification using selected features
forest_important = RandomForestClassifier(n_estimators=100, random_state=42)
forest_important.fit(X_important_train, y_train)





RandomForestClassifier(random_state=42)

In [22]:
from sklearn.metrics import classification_report, accuracy_score
# Validation prediction and evaluation
y_val_pred = forest_important.predict(X_important_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.962551103843009


In [23]:
# Predictions and evaluation selected
y_pred = forest_important.predict(X_important_test)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.94      0.98      0.96      9200
           1       0.98      0.94      0.96      9146

    accuracy                           0.96     18346
   macro avg       0.96      0.96      0.96     18346
weighted avg       0.96      0.96      0.96     18346

